In [1]:
from pathlib import Path
import yaml
import os

root_dir = Path("/media") / "sharedData" / "data"
root_name = "glass-a2744"

catalogue_dir = root_dir / "2024_08_16_A2744_v4" / "glass_niriss" / "match_catalogues"

grizli_home_dir = root_dir / "2024_08_16_A2744_v4" / "grizli_home"
grizli_extraction_dir = grizli_home_dir / "Extractions_v2"

In [2]:
from astropy.table import Table, join, Column, vstack
import numpy as np

v2_out_dir = grizli_home_dir / "classification-stage-2"/"catalogues" / "compiled"
cat_others = Table.read(v2_out_dir  / "internal_stage_3_2.fits")

stage_3_dir = grizli_home_dir / "Extractions_v3"
stage_3_cat = Table.read(stage_3_dir / "classification" / "stage_3_PJW_class.fits")

force_dir = grizli_home_dir / "ForcedExtractions"
detect_cat = Table.read(force_dir / f"{root_name}-ir.cat.fits")

stage_4_dir = grizli_home_dir / "Extractions_v4"
(stage_4_dir / "catalogues").mkdir(exist_ok=True, parents=True)

stage_6_cat = Table.read( stage_4_dir / "catalogues" / "stage_4_input.fits")
stage_6_class_cat = Table.read( stage_4_dir / "classification" / "stage_4_PJW_class.fits")

for s in stage_6_cat[(stage_6_cat["Z_NIRISS"]>=4.5) & (stage_6_cat["Z_NIRISS"]<=7.)]:
    print (*s["ID","RA","DEC","Z_FLAG"][0:])

172 3.600693377491722 -30.411901503025792 4
325 3.606861869664862 -30.41020274538738 4
394 3.60454558502696 -30.409342905610657 3
486 3.620440179634046 -30.407355253912947 4
1023 3.619032661340116 -30.401622556070002 4
1907 3.5979423626583964 -30.391859086545452 4
1930 3.6102533508913934 -30.39163595048282 4
2345 3.6042529573788054 -30.387235206722373 4
2386 3.595428713844152 -30.386792159279857 4
2528 3.61001960072486 -30.38576198189488 4
2530 3.583500430889578 -30.385742465057046 3
2535 3.610247199029373 -30.38558764021113 4
3240 3.6070606088253854 -30.37842111166278 4
3320 3.601427307912008 -30.376966046699057 3
3414 3.5998709843590735 -30.37470426717512 4


In [3]:
to_use = np.ones_like(stage_6_cat, dtype=bool)
for i, row in enumerate(stage_6_cat):
    class_idx = np.argwhere(row["ID"]==stage_6_class_cat["ID"].astype(int))
    if class_idx.size > 0:
        # print (class_idx, )
        class_data = stage_6_class_cat[class_idx]
        if "rm" in str(class_data["COMMENTS"]):
            to_use[i] = False
        elif class_data["UNRELIABLE_REDSHIFT"]:
            stage_6_cat[i]["Z_FLAG"] = 2
        elif class_data["TENTATIVE_REDSHIFT"]:
            stage_6_cat[i]["Z_FLAG"] = 3
    
    if row["Z_FLAG"]<=2:
        stage_6_cat[i]["Z_NIRISS"] = np.nan

    stage_6_cat[i]["Z_NIRISS"] = round(stage_6_cat[i]["Z_NIRISS"], 5)

new_extractions = (grizli_home_dir / "for_others" / "Peter").glob("**/*.full.fits")
from astropy.io import fits
for p in new_extractions:
    print (p)
    with fits.open(p) as hdul:
        obj_id = hdul[0].header["ID"]
        obj_z = round(hdul[1].header["Z_MAP"], 5)
        # print (hdul[1].data)
        print (obj_id, round(obj_z,5))
        class_idx = np.argwhere(stage_6_cat["ID"]==obj_id)[0,0]
        # print (class_idx)
        stage_6_cat[class_idx]["Z_NIRISS"] = obj_z
        stage_6_cat[class_idx]["Z_FLAG"] = 4
        # stage_6_cat[class_idx]["Z_NIRISS"] = round(obj_z,5)
        print (stage_6_cat[class_idx])
stage_6_cat = stage_6_cat[to_use]

stage_6_cat["Z_FLAG"][np.isin(stage_6_cat["ID"],[199,486,3593])] = 2
stage_6_cat["Z_NIRISS"][np.isin(stage_6_cat["ID"],[199,486,3593])] = np.nan

print(np.unique(np.array(stage_6_cat["Z_FLAG"]), return_counts=True))

stage_6_cat.write(stage_4_dir / "catalogues" / "stage_6_output_internal.fits", overwrite=True)
for s in stage_6_cat[(stage_6_cat["Z_NIRISS"]>=4.5) & (stage_6_cat["Z_NIRISS"]<=7.)]:
    print (*s["ID","RA","DEC","Z_FLAG"][0:])

/media/sharedData/data/2024_08_16_A2744_v4/grizli_home/for_others/Peter/60mas/full/glass-a2744_00380.full.fits
380 2.01409
 ID         RA                 DEC         Z_NIRISS Z_FLAG F115W,72.0_QUALITY F150W,72.0_QUALITY F200W,72.0_QUALITY F115W,341.0_QUALITY F150W,341.0_QUALITY F200W,341.0_QUALITY Z_OUTSIDE Z_ON_EDGE Z_REFIT        DZ_1PZ        NUMBER      MAG_AUTO    
                                                                                                                                                                                                                                               uJy       
--- ------------------ ------------------- -------- ------ ------------------ ------------------ ------------------ ------------------- ------------------- ------------------- --------- --------- ------- -------------------- ------ -----------------
380 3.5864256327693615 -30.409299110881705  2.01409      4                  1                  2                  2            

In [4]:
phot_cat = Table.read(force_dir / f"{root_name}_phot.fits")
phot_cat.rename_columns(phot_cat.colnames, [c.upper() for c in phot_cat.colnames])

stage_6_cat.remove_columns(["NUMBER","MAG_AUTO"])
stage_6_phot = join(stage_6_cat, phot_cat)
stage_6_phot.write(stage_4_dir / "catalogues" / "stage_6_output_internal_phot.fits", overwrite=True)
for s in stage_6_cat[(stage_6_cat["Z_NIRISS"]>=4.5) & (stage_6_cat["Z_NIRISS"]<=7.)]:
    print (*s["ID","RA","DEC","Z_FLAG"][0:])

172 3.600693377491722 -30.411901503025792 4
394 3.60454558502696 -30.409342905610657 3
1023 3.619032661340116 -30.401622556070002 3
1930 3.6102533508913934 -30.39163595048282 4
2530 3.583500430889578 -30.385742465057046 3
3320 3.601427307912008 -30.376966046699057 3
3414 3.5998709843590735 -30.37470426717512 3
